<img width="8%" alt="Sales" src="https://naasai-public.s3.eu-west-3.amazonaws.com/abi-demo/sales_conversion.png" style="border-radius: 15%">

# Sales plugin

**Tags:** #naaschatplugin #naas #naas_driver #chat #plugin #ai

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel)

**Description:** This notebook creates a Naas Chat plugin with commands.

## Input

### Import libraries

In [ ]:
import naas
from naas_drivers import naas_chat_plugin, gsheet
import naas_data_product
from IPython.display import Markdown
from datetime import date, timedelta
import os
import pandas as pd

### Setup variables
**Inputs**
- `entity_dir`: Entity directory.
- `input_dir`: Input directory to retrieve file from.
- `input_file`: Input file.
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name`: Google Sheets sheet name.
- `name`: Plugin name.
- `model`: Plugin chat model.
- `temperature`: Plugin chat temperature.
- `description`: Plugin description.
- `avatar`: Plugin avatar image.
- `model_dir`: Models directory.

**Outputs**
- `output_dir`: Output directory where output files are stored. 
- `entity_name`: Entity name.

In [ ]:
# Inputs
entity_id = "0"
entity_dir = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", entity_id), "entity_dir")
input_dir = os.path.join(entity_dir, "sales-engine", date.today().isoformat())
input_file = "deals"
spreadsheet_url = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", entity_id), "abi_spreadsheet")
sheet_name = "DEALS"
name = "⚡️ Sales Assistant"
model = "gpt-4-1106-preview"
temperature = 0.2
description = "Nurture your sales qualified leads and ensure they progress through the sales pipeline effectively."
avatar = "https://naasai-public.s3.eu-west-3.amazonaws.com/abi-demo/sales_conversion.png"
model_dir = os.path.join(naas_data_product.ROOT_PATH, "models", "sales-engine")
input_image = "deals_trend.png"
input_html = "deals_trend.html"

# Outputs
output_dir = os.path.join(entity_dir, "plugins")
entity_name = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", entity_id), "entity_name")

## Model

### Get assets

In [ ]:
image_url = get_image_asset(input_dir, input_image)
print("Image URL:", image_url)

image_html = get_image_asset(input_dir, input_html)
print("Image HTML:", image_html)

### Get data to be sent to prompt

#### Get sales qualified leads

In [ ]:
# Init
sales_data = ""
new_deals = 0

# Load data from pickle
data = pload(input_dir, input_file)    
if data is None:
    data = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name)
if isinstance(data, pd.DataFrame):
    df = data.copy()

# Filter on this week and last week
if len(df) > 0:
    df = df[(df["ENTITY"].str.contains(entity_name)) & (df["SCENARIO"].isin([TW, LW]))]
    
    # Get data
    tmp_df = df[(df["SCENARIO"].isin([TW]))]
    new_deals = len(tmp_df)
    if len(df) > 0:
        limit = 0.45
        for row in df.itertuples():
            index = row.Index
            prompt_tokens, max_tokens = naas_chat_plugin.check_tokens(
                prompt=df[:index+1].to_string(),
                model=model,
                limit=limit,
            )
            if prompt_tokens > max_tokens * limit:
                break
        df = df[:index+1]
    
sales_data = df.to_string()

print()
print(f"- New Deals: {new_deals}")

### Create system prompt

In [ ]:
system_prompt = f"""
Act as a Sales Assistant who has access to a list of deals generated thanks to your contents published on social media platform. 
Your role is to manage and nurture theses deals, ensuring that they progress through the sales pipeline effectively.
Begin your conversation by introducing yourself and displaying the image: [![Deals Evolution]({image_url})({image_html})] inside the markdown of the chat that illustrates the evolution of deals over the past two weeks, thanks to the content published by '{entity_name}'. 
Next, provide an update on this week's results: 
During this week '{TW}', the content from '{entity_name}' has generated {new_deals} new promising deals.
Proceed with a brief analysis of the new deals made this week '{TW}' starting with the deal name and giving details of the people and organization involved in each deal: {sales_data}.
For the convenience of your sales team, please hyperlink the people fullname and organization name to their respective LinkedIn URL.
While maintaining a casual yet professional tone, ensure to stick to the requested information and avoid adding unnecessary details.
After receiving the user's first response, you can then commence with the first high-level analysis.
"""

### Create commands

In [ ]:
commands = create_command(model_dir)

## Output

### Create Naas Chat plugin
This function will generate the plugin in JSON format and also verify if your prompt adheres to the recommended limit, which is set at 20% of the maximum tokens allowed by the model. Then, it will save your plugin in your local environment.

In [ ]:
output_path = os.path.join(output_dir, f"{name.lower().replace(' ', '_')}.json")

# Create plugin
plugin_file_path = naas_chat_plugin.create_plugin(
    name=name,
    prompt=system_prompt.replace("\n", ""),
    model=model,
    temperature=temperature,
    output_path=output_path,
    description=description,
    avatar=avatar,
    commands=commands
)

### Create asset
This asset can be utilized by using the command `/use` in your Naas Chat or by simply clicking on the link provided in the last cell output.

In [ ]:
plugin_url = naas.asset.add(plugin_file_path, params={"inline": True})
print(plugin_url)

### Create new chat
NB: You don't need to click on 'Create New Chat' everytime you update your system prompt, you can use the command `/refresh`.

In [ ]:
Markdown(f"[Create New Chat](https://naas.ai/chat/use?plugin_url={plugin_url})")